# Model training

## Multinomial logistic regression

When there is only 2 possible outcomes for the target, it is```binomial```.

```Multinomial``` : The target variable has three or more possible classes.
Indeed, there is a discrete number of possible outcomes = ['Ravenclaw', 'Slytherin', 'Gryffindor', 'Hufflepuff'].

```One-Vs-All Classification``` is a method of multi-class classification.
Braking down by splitting up the multi-class classification problem into `multiple binary classifier models`.

in One-vs-All multi-class classification :
For k = 4 class labels present in the dataset, k = 4  ```binary classifiers``` are needed.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset_train = f'../datasets/dataset_train.csv'
df = pd.read_csv(dataset_train)
outcomes = df['Hogwarts House'].unique().tolist()
outcomes

## Trainset data processing

### Selecting features and rows of interest 
- pd.drop() : down to to 10 meaningful features, independent from each other
- pd.dropna() : Dropping rows that contain NaN => down to 1333 rows

In [ ]:

df.drop(df.columns[2:6], inplace=True, axis = 1)
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df.drop(excluded_features, inplace=True, axis=1)
df.dropna(inplace=True)
df.head()

### Standardizing dataset
- standardizing values for each feature, apply along axis=1, using the `z-score method`.

The z-score method (often called standardization) transforms the info into distribution with a mean of 0 and a typical deviation of 1. Each standardized value is computed by subtracting the mean of the corresponding feature then dividing by the quality deviation.



In [ ]:
def standardize(arr: np.ndarray):
    """z-score method"""
    mean = np.mean(arr)
    std = np.std(arr)
    return (arr - mean) / std

df_class = df['Hogwarts House'].copy(deep=True)
df_train= df.drop(df.columns[:2], axis = 1)
df_std_train = df_train.agg(lambda course: standardize(course))
df_std_train.head()


In [ ]:
df_std_train['Real Output'] = df_class
df_std_train.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 5, figsize=(20, 20))
features = df_std_train.columns[:-1].to_list()
for idx in range(10):
    i = idx // 5
    j = idx % 5
    sns.boxplot(data=df_std_train, x='Real Output', ax=axs[i, j], y=features[idx])
plt.show()

### one-vs-all :
actual class y set to 1 or 0

- 1 = is in house,
- 0 = is in another house

In [ ]:
houses = df['Hogwarts House'].unique()
houses[0]
# gives bool : df['Hogwarts House'] == houses[0]
y_actual = np.where(df['Hogwarts House'] == houses[0], 1, 0)
y_actual

## Data training

Excellent explanantions here :
https://www.kaggle.com/code/sugataghosh/implementing-logistic-regression-from-scratch

### Intializing

- standardized data to a numpy array 
- a column of ones added to the left for the `ìntercept` or bias

for each `classifier` : 
Values of 𝑛 features $X =(𝑥_1,𝑥_2,⋯,𝑥_𝑛)$ 

#### Dot product :

Introduced Weights  $W =(𝑤_1,𝑤_2,⋯,𝑤_𝑛)$ so that $z = 𝑏 + 𝑥_1.𝑤_1 + 𝑥_2.𝑤_2 + ... +𝑥_n.𝑤_n$,
𝑏 being the bias parameter.

Basically, the dot product of inputs and weights
$$
\mathbf{X} \cdot \mathbf{W} = \sum_{i=1}^n 𝑥_i 𝑤_i
$$

$\mathbf{z} = b + \mathbf{X} \cdot \mathbf{W} =$ is feeding the logistic function 𝑔, and projects the output as the predicted probability of 𝑦 being equal to 1.

$$
y = g(z) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{-(X.W + 𝑏)}}
$$

#### Weights

Transposed matrix of zeros, 
Shape : (one intercept + number of features) x (number of k classifiers)

$$
\begin{pmatrix}
\ b & \ 𝑤_1 & \  𝑤_2 & \ ⋯ & \ 𝑤_𝑛 \\ 
\end{pmatrix}
$$

### Input : X Array

a column of ones is added to x_train array so that the bias is multiplied by 1.

$$
\begin{pmatrix}
\ 1 \\
\ x_1 \\
\ 𝑥_2 \\
\ ⋯ \\
\ 𝑥_𝑛 \\
\end{pmatrix}
$$

### Dot product for one output 
$$
\mathbf{z} = 
\begin{pmatrix}
\ 1 \\
\ x_1 \\
\ 𝑥_2 \\
\ ⋯ \\
\ 𝑥_𝑛 \\
\end{pmatrix}
\cdot
\begin{pmatrix}
\ b & \ 𝑤_1 & \  𝑤_2 & \ ⋯ & \ 𝑤_𝑛 \\ 
\end{pmatrix}
=  𝑏 + 𝑥_1.𝑤_1 + 𝑥_2.𝑤_2 + ... +𝑥_n.𝑤_n
$$


In [ ]:
""" Parameters : unstandardized data to train without NaN, output """
df_std_train = df_train.agg(lambda course: standardize(course))
x_train_std = np.array(df_std_train)
ones = np.ones((len(x_train_std), 1), dtype=float)
x_train = np.concatenate((ones, x_train_std), axis=1)
features = df_std_train.columns.tolist()
df_class = df['Hogwarts House'].copy(deep=True)
houses = df_class.unique().tolist()
w_indexes = df_std_train[:-1].columns.insert(0, ['Intercept'])
df_weights = pd.DataFrame(columns=houses, index=w_indexes).fillna(0)
df_weights.head(11)

In [ ]:
df_std_train.head()

Sigmoid function (or logistic function) to map input values from a wide range into a limited interval. 
$sigmoid function$
$$
y = g(z) = \frac{1}{1 + e^{-z}} = \frac{e^z}{1 + e^z}
$$
This formula represents the probability of observing the output y = 1 of a Bernoulli random variable. This variable is either 1 or 0 :
$$
y \in \{0,1\}
$$

In [ ]:
def sigmoid(arr: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-arr))

def update_weight_loss(weights, learning_rate, grad_desc):
    return weights - learning_rate * grad_desc

def train_one_vs_all(house, df_class, features, x_train, learning_rate, epochs):
    """
    loss_iter = LogRegTrain.loss_function(y_actual, h_pred)
    gradient = np.dot(x_train.T, (h_pred - y_actual))
    """
    y_actual = np.where(df_class == house, 1, 0)
    weights = np.ones(len(features) + 1).T
    for iter in range(epochs):
        z_output = np.dot(x_train, weights)
        h_pred = sigmoid(z_output)
        tmp = np.dot(x_train.T, (h_pred - y_actual))
        grad_desc = tmp / y_actual.shape[0]
        weights = update_weight_loss(weights, learning_rate, grad_desc)
    return weights


learning_rate = 0.1
epochs = 1000
for house in houses:
    weights = train_one_vs_all(house, df_class, features, x_train, learning_rate, epochs)
    df_weights[house] = weights
print("alpha = ", learning_rate, "  iterations =", epochs)
df_weights.head(11)

STOPS HERE

In [ ]:
def loss(y_actual, h_pred):
    """ y_actual : target class. 1 in class, 0 not in class
    h_pred = signoid(x.weights)
    loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    """
    m = len(h_pred)
    a = - y_actual * np.log(h_pred)
    b = (1 - y_actual) * np.log(1 - h_pred)
    return (a - b) / m

loss(y_actual, h_pred)


The weights are updated by substracting the derivative (gradient descent) times the learning rate,
loss'(theta) = 
def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]
def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

In [ ]:
v1_m = np.ma.array(x_train, mask=np.isnan(x_train))
res = (h_pred - y_actual)
v2_m = np.ma.array(res, mask=np.isnan(res))
#dot = np.ma.dot(x_train, v2_m.T)
dot = np.ma.dot(v1_m.T, v2_m)
gradient1 = dot / y_actual.shape[0]
gradient1

replace np.nan with zeros
```x_train[np.isnan(x_train)] = 0 ```

In [ ]:
dot = np.dot(x_train.T, (h_pred - y_actual))
gradient = dot / y_actual.shape[0]
gradient

In [ ]:
def gradient_descent(x_train, h_pred, y_actual):
    return np.dot(x_train.T, (h_pred - y_actual)) / y_actual.shape[0]

def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

gd =gradient_descent(x_train, h_pred, gradient)
print("gd = ", gd)
weights = update_weight_loss(weights, 0.1, gradient_descent(x_train, h_pred, y_actual))
print(" w =", weights)



The weights are updated by substracting the derivative (gradient descent) times the learning rate,